

$f(x)=x^5-8x^3+10x+6$

[-3,3] 구간 내

In [27]:
import numpy as np

In [178]:
f=lambda x: x**5-8*x**3+10*x+6
df=lambda x:5*x**4-24*x**2+10

In [179]:
f(1)

9

In [243]:
# 초기 구간 설정을 위한 bracket_minimum algorithm

def bracket_minimum(f,x,s=1E-2,k=2.0): 
    a,ya=x,f(x)
    b,yb=a+s,f(a+s)
    
    #print('init: (a:%.4f, b:%.4f) (ya:%.4f, yb:%.4f)' %(a,b,ya,yb))
    
    if yb>ya:
        a,b=b,a #스와핑
        ya,yb=yb,ya
        s=-s #b 값 줄여나감
    
    while True:
        c,yc=b+s, f(b+s)
        #print('step: (a: %.4f, b:%.4f, c:%.4f) (ya:%.4f, yb:%.4f, yc:%.4f)' %(a,b,c,ya,yb,yc))
        
        if yc>yb:
            if a<-3:
                a=-3 #구간의 하한 설정
            elif c>3: 
                c=3 #구간의 상한 설정
            return (a,c) if a<c else (c,a)
            # while 루프 빠져나옴
        else:
            a,ya,b,yb=b,yb,c,yc
            s*=k #더 넓은 구간으로 이동

In [244]:
print('--- x변경 ---')
print('x=-1: ',bracket_minimum(f,-1,s=1E-8,k=2.0))
print('x=0: ',bracket_minimum(f,0,s=1E-8,k=2.0))
print('x=1: ',bracket_minimum(f,1,s=1E-8,k=2.0))
print('x=2: ',bracket_minimum(f,2,s=1E-8,k=2.0))

print('--- s변경 ---')
print('s=0.01: ',bracket_minimum(f,0,s=1E-2,k=2.0))
print('s=0.001: ',bracket_minimum(f,0,s=1E-3,k=2.0))

print('--- k변경 ---')
print('k=1.5: ',bracket_minimum(f,1,s=1E-2,k=1.5))
print('k=2.0: ',bracket_minimum(f,1,s=1E-3,k=2.0))
print('k=2.5: ',bracket_minimum(f,1,s=1E-3,k=2.5))
print('k=2.0: ',bracket_minimum(f,1,s=1E-3,k=3.0))

--- x변경 ---
x=-1:  (-0.8322278399999999, -0.3289113599999999)
x=0:  (-1.34217727, -0.33554431)
x=1:  (1.6710886399999998, 3)
x=2:  (2.0419430399999996, 2.1677721599999997)
--- s변경 ---
s=0.01:  (-1.27, -0.31000000000000005)
s=0.001:  (-1.0230000000000001, -0.255)
--- k변경 ---
k=1.5:  (1.7588671875, 2.719951171875)
k=2.0:  (1.5119999999999998, 3)
k=2.5:  (1.4072343749999998, 3)
k=2.0:  (1.3649999999999995, 3)


# 삼분할 탐색법


In [245]:
def trifold_search(f,x,epsilon=1E-6):
    a,b=bracket_minimum(f,x)
    #print('init: (a:%.4f, b:%.4f)' %(a,b))
    
    distance=abs(a-b)
    
    i=1
    while distance>epsilon:
        x1=a+(1.0/3.0)*distance
        x2=a+(2.0/3.0)*distance
        
        y1,y2=f(x1),f(x2)
        
        if y1>y2:
            a,b=x1,b
            
        else:
            a,b=a,x2
            
            
        distance=abs(a-b)
        
        #print('%d: (a:%.4f, b:%.4f)' %(i,a,b))
        i+=1
    
    x=0.5*abs(a+b)
    y=f(x)
    
    return x,y,i

In [237]:
trifold_search(f,1)

(2.0830437420086856, -6.258776371634173)

In [246]:
print('--- x변경 ---')
for i in [-1,0,1,2]:
    print('x=',i,end="\t")
    print(trifold_search(f,i))

print()
    
print('--- epsilon변경 ---')

for e in [1E-4,1E-6,1E-8]:
    print('epsilon=',e,end='\t')
    print(trifold_search(f,1,epsilon=e))

--- x변경 ---
x= -1	(0.678916665067644, 10.429952806772517, 34)
x= 0	(0.678916767353898, 10.429952806772814, 35)
x= 1	(2.0830437420086856, -6.258776371634173, 36)
x= 2	(2.083044059532636, -6.2587763716344895, 30)

--- epsilon변경 ---
epsilon= 0.0001	(2.083045801313978, -6.258776371491308, 25)
epsilon= 1e-06	(2.0830437420086856, -6.258776371634173, 36)
epsilon= 1e-08	(2.0830439126465743, -6.258776371635349, 48)


# 피보나치 탐색법

In [247]:
def fibonacci_search(f,x,n,epsilon=1E-2):
    a,b=bracket_minimum(f,x) #구간의 상한 하한 결정하는 bracket_minimum 알고리즘 활용
    #print('init: (a:%.4f, B:%.4f)' % (a,b))
    
    psi=0.5*(1.+np.sqrt(5))
    s=(1.-np.sqrt(5))/(1.+np.sqrt(5))
    
    rho=1./psi*((1.-s**(n+1))/(1.-s**n)) #n은 1,1,2,3,5,8에서 몇번째인지 결정    
    d=rho*b+(1.-rho)*a
    
    yd=f(d)
    
    for i in range(1,n):
        if i==n-1:
            c=epsilon*a+(1.-epsilon)*d
        else:
            c=rho*a+(1.-rho)*d
        yc=f(c)
        
        if yc<yd:
            b,d,yd =d,c,yc
            
        else:
            a,b=b,c
            
        rho=1./psi*((1.-s**(n-i+1)))/(1.-s**(n-i))
        
        pa,pb=(a,b) if a<b else (b,a)
        #print('%d:(a:%.4f, b:%.4f)' %(i,pa,pb))
        
    a,b=(a,b) if a<b else (b,a)
    
    x=0.5*abs(a+b)
    y=f(x)

    return x,y,i

In [248]:
print('--- x값 변경---')
for i in [-2,-1,0,1,2]:
    print('x=',i,end="\t")
    print(fibonacci_search(f,i,n=50))
    
print()

print('--- n 값 변경 ---')
for n in [30,50,100]:
    print('n=',n,end='\t')
    print(fibonacci_search(f,1,n=n))
    
print() 
print('--- epsilon변경 ---')
for e in [1E-4,1E-6,1E-8]:
    print('epsilon=',e,end='\t')
    print(fibonacci_search(f,1,n=50,epsilon=e))

--- x값 변경---
x= -2	(0.6789185461920191, 10.429952806733919, 49)
x= -1	(0.6789172605329832, 10.429952806770377, 49)
x= 0	(0.6789170856339406, 10.429952806771974, 49)
x= 1	(2.0830452604511756, -6.258776371562007, 49)
x= 2	(2.083044107369532, -6.258776371633822, 49)

--- n 값 변경 ---
n= 30	(2.083169294503794, -6.258775736585431, 29)
n= 50	(2.0830452604511756, -6.258776371562007, 49)
n= 100	(2.0830439193429777, -6.258776371635356, 99)

--- epsilon변경 ---
epsilon= 0.0001	(2.0830452604511756, -6.258776371562007, 49)
epsilon= 1e-06	(2.0830452604511756, -6.258776371562007, 49)
epsilon= 1e-08	(2.083039653722424, -6.258776370902638, 49)


# 황금 분할 탐색법

In [227]:
def golden_section_search(f,x,epsilon=1E-6): 
    # f는 목적함수, x는 bracket minimum에 의해 feasible set을 찾기 위한 초기값, epsilon은 허용오차
    
    a,b=bracket_minimum(f,x)
    #print('init:(a:%.4f, b:%.4f)' %(a,b))
    
    distance=abs(a-b)
    
    psi=0.5*(1.+np.sqrt(5))
    rho = psi**(-1) # 구간을 결정해주는 convex combination하기 위해 rho 계산: 황금비율의 역수
    
    d=rho*b+(1.-rho)*a
    yd=f(d)
    
    i=1
    #구간의 길이가 허용오차보다 작아지면 반복 멈춤-> 책에서는 n을 주어졌는데 우리는 epsilon으로 해결1
    while distance>epsilon:
        
        c=rho*a+(1.-rho)*b #같은 방법의 convex combination이용
        yc=f(c)
        
        if yc<yd:
            b, d,yd =d, c,yc
            
        else:
            a,b=b,c
            
        pa,pb=(a,b) if a<b else(b,a)
        #print('%d:(a:%.4f, b:%.4f)'%(i,pa,pb))
        
        distance=abs(a-b)
        
        i+=1
    
    a,b=(a,b) if a<b else(b,a)
    x=0.5*(a+b)
    y=f(x)

    
    return x,y

In [228]:
print('--- x값 변경---')
for i in [-2,-1,0,1,2]:
    print('x=',i,end="\t")
    print(golden_section_search(f,i))
    
print()

print('--- epsilon변경 ---')
for e in [1E-4,1E-6,1E-8]:
    print('epsilon=',e,end='\t')
    print(golden_section_search(f,1,epsilon=e))

--- x값 변경---
x= -2	(-0.6789167640439839, 1.5700471932271922)
x= -1	(-0.6789168754193231, 1.5700471932271718)
x= 0	(-0.6789168903459053, 1.570047193227194)
x= 1	(2.083043848613427, -6.258776371635193)
x= 2	(2.083044114450747, -6.258776371633719)

--- epsilon변경 ---
epsilon= 0.0001	(2.0830567397569197, -6.2587763649896715)
epsilon= 1e-06	(2.083043848613427, -6.258776371635193)
epsilon= 1e-08	(2.0830439204698683, -6.258776371635349)


# 이분법

In [229]:
def bracket_sign_change(df,a,b,k=2): 
    #df는 목적함수(x), 도함수(o) , a와 b는 가까운 값, k는 하이퍼 파라미터(width를 늘려나가는 비율)
    # a,b,k값이 초모수, 하이퍼 파라미터
    
    if a>b:
        a,b=b,a
        
    center,half_width = 0.5*(b+a), 0.5*(b-a)
    
    while df(a)*df(b)>0:
        
        half_width*=k
        
        a=center-half_width
        b=center+half_width
        
    return (a,b)

In [230]:
print('--- a,b값 변경---')
a_lst=[-1.5,-1,0,1.5,1]
b_lst=[-1,-1.5,0.5,2,1.7]
for a,b in zip(a_lst,b_lst):
    print('a=',a,end="\t")
    print('b=',b,end="\t")
    print(bracket_sign_change(df,a,b))
    
print()

print('--- k값 변경 ---')
for k in [1.5,2.0,2.5,3.0]:
    print('k=',k,end='\t')
    print(bracket_sign_change(df,0,0.5,k=k))

--- a,b값 변경---
a= -1.5	b= -1	(-3.25, 0.75)
a= -1	b= -1.5	(-3.25, 0.75)
a= 0	b= 0.5	(-0.25, 0.75)
a= 1.5	b= 2	(1.25, 2.25)
a= 1	b= 1.7	(0.6500000000000001, 2.05)

--- k값 변경 ---
k= 1.5	(-0.3125, 0.8125)
k= 2.0	(-0.25, 0.75)
k= 2.5	(-0.375, 0.875)
k= 3.0	(-0.5, 1.0)


In [231]:
def bisection(df,x,epsilon=1E-6): 
    # 목적함수가 아닌 도함수 들어가야함!, 
    # 이분법에 의해 찾아진 구간 길이가 epsilon보다 작으면 반복 멈춘다
    # x는 bracket_sing_change를 a,b를 결정하기 위해 임의로 지정해줌
    
    a,b=bracket_sign_change(df,x-epsilon,x+epsilon)
    #print('init:(a:%.4f, b:%.4f)'%(a,b)) #feasible set지정
    
    ya,yb=df(a),df(b)
    
    if ya==0:
        b=a
    if yb==0:
        a=b
        
    i=1
    while b-a > epsilon: #b-a는 구간의 길이: b가 항상 a보다 크니까
        x=0.5*(a+b)
        y=df(x)
        
        if y==0:
            a,b=x,x
        elif y*ya>0: #두 도함수의 부호가 같다 -> a점을 x로 이동
            a=x
        else: #그렇지 않으면 b를 x로 이동
            b=x
            
        #print('step %d -a:%.4f, b:%.4f, y:%.4f, ya:%.4f' %(i,a,b,y,ya))
        
        i+=1
        
        x=0.5*(a+b)
        y=df(x)

        
    return x,y

In [232]:
print('--- x값 변경 ---')
for x in [-2,-1,1,2]:
    print('x=',x,end='\t')
    print(bisection(df,x))
    
print()

print('--- epsilon값 변경 ---')
for e in [1E-2,1E-4,1E-6,1E-8]:
    print('epsilon=',e,end='\t')
    print(bisection(df,x=0.5,epsilon=e))

--- x값 변경 ---
x= -2	(-2.083043500002388, -3.3353270112002065e-05)
x= -1	(-0.6789165000085907, 8.591050569606296e-06)
x= 1	(0.6789165000085907, 8.591050569606296e-06)
x= 2	(2.083043500002388, -3.3353270112002065e-05)

--- epsilon값 변경 ---
epsilon= 0.01	(0.6775000000000002, 0.03728368769530732)
epsilon= 0.0001	(0.6789499999999803, -0.0008734538571033568)
epsilon= 1e-06	(0.6789167500001789, 2.008930223951211e-06)
epsilon= 1e-08	(0.67891682540242, 2.3636673063265334e-08)
